In [ ]:
import requests
import time
import os
import numpy as np
import subprocess
from operator import itemgetter
import glob
import random
import fnmatch
import itertools
from bs4 import BeautifulSoup
from urllib.request import urlopen

repo_pathname = '/media/smcl/Audio/epc_pages/'

os.chdir(repo_pathname)

!ls

In [ ]:
import csv

urls_lol = []

row_set = set()

with open('EPC_Link_Status_Table.csv') as fi:
    csv_reader = csv.reader(fi)
    for row in csv_reader:
        if '|'.join(row[:2]) not in row_set:   ### Removing duplicates
            urls_lol.append(row)
            row_set.add('|'.join(row[:2]))

header = urls_lol[0]
urls_lol = urls_lol[1:]        

len(urls_lol)

In [ ]:
random.choice(urls_lol)

In [ ]:
import csv

urls_404 = []

for row in urls_lol:
    if row[2] in ('404', 'dead'):
        urls_404.append(row[0])

urls_404 = sorted(urls_404)
            
len(urls_404)

In [ ]:
from urllib.request import urlopen
import json
import random

urls = urls_404

random.shuffle(urls)

url_dict = {}

for url in urls_404:
    api_url = 'https://archive.org/wayback/available?url=' + url
    try:
        page = urlopen(api_url).read().decode('utf8')
    except Exception as e:
        print(e)
        print(api_url)
    json_object = json.loads(page)
    try:
        wayback_url = json_object['archived_snapshots']['closest']['url']
        print(wayback_url)
        url_dict[url] = wayback_url
    except:
        try:
            api_url = 'https://archive.org/wayback/available?url=' + json_object['url']
            page2 = urlopen(api_url).read().decode('utf8')
            json_object = json.loads(page2)
            wayback_url = json_object['archived_snapshots']['closest']['url']
            print(wayback_url)
            url_dict[url] = wayback_url
        except:
            print(page)

In [ ]:
!cp EPC_Link_Status_Table.csv EPC_Link_Status_Table_backup.csv

In [ ]:

urls_301 = []

for row in urls_lol:
    if row[3]!='':
        urls_301.append(row[0])

urls_301 = sorted(urls_301)
            
len(urls_301)

In [ ]:
import requests

url_301_dict = {}

for url in urls_301:
    try:
        r = requests.head(url)
        if r.status_code in (301, 302, 303, 304):
            url_301_dict[url] = r.headers['Location'] ######## <-------------------
    except Exception as e:
        print(e)

In [ ]:
for key in url_301_dict:
    if url_301_dict[key] != key:
        print(key)
        print(url_301_dict[key])
        print('---')

In [ ]:
#header

header_2 = ['url', 'linked from', 'server response', 'redirect link', 'replacement link']

header_2

In [ ]:
urls_lol_2 = []

for row in urls_lol:
    temp_row = row + ['']
    if row[3]!='':
        try: temp_row[3] = url_301_dict[row[3]]
        except: pass
    if row[2] in ('404', 'dead'):
        try: temp_row[4] = url_dict[row[0]]
        except: pass
    urls_lol_2.append(temp_row)

In [ ]:
random.sample(urls_lol_2, 10)

In [ ]:
import csv

with open('EPC_Link_Status_Table.csv', 'w') as fo:
    csv_writer = csv.writer(fo)
    csv_writer.writerow(header_2)
    csv_writer.writerows(urls_lol_2)

In [ ]:
!wc -l 'EPC_Link_Status_Table.csv'